In [1]:
import numpy as np
import math
from scipy.sparse import csr_matrix

In [8]:
class FiniteDifferences:

  def __init__(self, x, f, h):
    self.x = x    # X nodal points
    self.f = f    # function f
    self.M = len(x)
    self.h = h    # Step

  def create_stiffness_matrix(self, p, q, boundary_conditions):
    A = csr_matrix((self.M-2, self.M-2), dtype=np.float64).toarray()    # Triagonal matrix
    F = np.zeros(self.M-2, dtype=np.float64)    # F vector for function f
    U = np.zeros(np.size(F), dtype=np.float64)    # Solution of the linear system
    h = self.h ** 2.    # Step

    # Create triagonal matrix
    for i in range(0, self.M-2):
      for j in range(0, self.M-2):
        if i == j:
          A[i][j] = (2. * p(_)) + h*q(_)
        elif i == j-1 or i == j+1:
          A[i][j] = -1. * p(_)
        else:
          A[i][j] = 0.

    # Set boundary conditions to vector F
    F[0] = h * self.f(self.x[1]) + boundary_conditions[0]
    F[self.M-3] = h * self.f(self.x[self.M-2]) + boundary_conditions[1]

    for i in range(1, self.M-3):
      F[i] = h * self.f(self.x[i+1])
    
    return A, F, U

  def gauss_elimination(self,A, F):
    n = len(F) #n is matrix size

    #Elimination phase
    for k in range(0,n-1): #k is matrix row
        for i in range(k+1,n): #i is matrix col
                  if A[i,k] != 0.0:
                    factor = A[i,k]/A[k,k]
                    A[i,k+1:n] = A[i,k+1:n] - np.multiply(factor,A[k,k+1:n])
                    F[i] = F[i] - np.multiply(factor,F[k])

    #Back substitution
    for k in range(n-1,-1,-1):
          F[k] = (F[k] - np.dot(A[k,k+1:n],F[k+1:n]))/A[k,k]

    return F


  def solve(self, p, q, boundary_conditions):
    A, F, initial_U = self.create_stiffness_matrix(p, q, boundary_conditions)

    # Solve linear system
    F = F[:, np.newaxis]
    initial_U = initial_U[:, np.newaxis]

    U_Gauss = self.gauss_elimination(A, F)

    return U_Gauss, A, F

  # Method to calculate the error
  def error(self, err_arr):
    error_sum = 0
    for i in range(1, len(self.x)-1):
      error_sum += self.h * (abs(err_arr[i-1]) ** 2.)

    return error_sum ** 1./2.

  def compare_solutions(self, U, u):
    u_exact = []    # Array to store exact solutions at nodal points 'xps'
    u_fd = []      # Array to store approximation solutions at nodal points 'xps'
    error_arr = []    # Array to store local absolute error 
    U = np.asarray(U)

    for i in range(1, len(self.x)-1):
      u_exact.append(u(self.x[i]))    # Calculate the exact solution u
      u_fd.append(U[i-1][0])          # Get the approximated solution U
      error_arr.append(abs(u_fd[i-1] - u_exact[i-1]))   # Calculate the absolute local error
        
    return u_exact, u_fd, self.error(error_arr), error_arr    

In [10]:
# Interval
interval = [0,2]  

# Subintervals
J = 100

# h is step size
h = 0.00005
#h = (interval[1] - interval[0])/J   # Step size. Reducing the step we get better approximation

# functions
phi = 10. * math.pi
u = lambda x: math.cos(phi * x)
q = lambda x: 2. * (phi ** 2.)
p = lambda x: 1.
b = math.cos(2. * phi)
f = lambda x: 3. * (phi**2.) * math.cos(phi * x)

# Initial x nodal points
x_points = np.arange(interval[0], interval[1] + h, h)     

# Boundary conditions
boundary_conditions = np.array([1., b], dtype=np.float64) 


model = FiniteDifferences(x_points, f, h)

# U is the solution to the linear system
# A is the triagonal matrix
# F is the vector of the function f
U, A, F = model.solve(p, q, boundary_conditions)


# Evaluate U solution with the exact solution
u_exact, fd, err, err_arr = model.compare_solutions(U, u)


# Uncomment to compare the exact solution with the approximated 
"""
for i in range(len(err_arr)):
  print(f'{u_exact[i]}     ==>     {fd[i]}     {err_arr[i]}\n')
"""
print(f'\nError: {err}')



KeyboardInterrupt: 